> <p><small><small>Copyright 2021 DeepMind Technologies Limited.</p>
> <p><small><small> Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at </p>
> <p><small><small> <a href="https://www.apache.org/licenses/LICENSE-2.0">https://www.apache.org/licenses/LICENSE-2.0</a> </p>
> <p><small><small> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License. </p>


# Generative Models


**Aim**

Our goal is to introduce you, students interested in STEM and or Artificial Intelligence, to the fascinating world of generative models; algorithms that are able to generate realistic novel creations, such as texts, music or, as in this colab, images.

**Disclaimer**

This code is intended for educational purposes, and in the name of readability for a non-technical audience does not always follow best practices for software engineering.

**Links to resources**
- [What is Colab?](https://colab.research.google.com) If you have never used Colab before, get started here!

# Can you tell which of these images are real?

Let's play a game. We'll show you some images, some of them have been generated using Artificial Intelligence (AI), some of of them are real.

Will you be able to spot the difference?

First, let's set up the quiz by running the cell below:


In [ ]:
#@title Setting up the "Real or Fake?" quiz.
#@markdown This may take a minute. 
%%capture
# Add all the pip installs of modules necessary to run the colab
%reset -f

!apt-get update
!apt-get install pip

# You can directly pip install packages
# e.g. !pip install dm-acme
# or you can clone the repo from GithHub
# e.g. !git clone https://github.com/deepmind/acme
!pip install ipywidgets


#License for the emoji lib: https://github.com/carpedm20/emoji/blob/master/LICENSE.txt
!pip install emoji --upgrade

# Import all the modules needed to run the colab here 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


import io
import IPython.display
import math
from matplotlib import pylab
from matplotlib import pyplot as plt
from matplotlib import rc
rc('animation', html='jshtml')
import matplotlib.animation as animation
import numpy as np
import PIL.Image
from scipy.stats import truncnorm
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
from google.colab import widgets
import time
from skimage import io
import ipywidgets
import emoji

# Set up the quiz.

#Create the GAN source of data. Copied from https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/biggan_generation_with_tf_hub.ipynb#scrollTo=ry62-8SWfuds
# BigGAN (original) models
tf.reset_default_graph()
module_path = 'https://tfhub.dev/deepmind/biggan-128/2'  # 128x128 BigGAN
module = hub.Module(module_path)
inputs = {k: tf.placeholder(v.dtype, v.get_shape().as_list(), k)
          for k, v in module.get_input_info_dict().items()}
output = module(inputs)

# Helper functions for BigGAN. Copied from https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/biggan_generation_with_tf_hub.ipynb#scrollTo=ry62-8SWfuds
input_z = inputs['z']
input_y = inputs['y']
input_trunc = inputs['truncation']

dim_z = input_z.shape.as_list()[1]
vocab_size = input_y.shape.as_list()[1]

def truncated_z_sample(batch_size, truncation=1., seed=None):
  """Draws noise samples from a truncated Gaussian distribution."""
  state = None if seed is None else np.random.RandomState(seed)
  values = truncnorm.rvs(-2, 2, size=(batch_size, dim_z), random_state=state)
  return truncation * values

def one_hot(index, vocab_size=vocab_size):
  """Creates one-hot vectors to encode the label (or category).

  e.g. one_hot(2, 5) = [0, 0, 1, 0, 0].
  """
  index = np.asarray(index)
  if len(index.shape) == 0:
    index = np.asarray([index])
  assert len(index.shape) == 1
  num = index.shape[0]
  output = np.zeros((num, vocab_size), dtype=np.float32)
  output[np.arange(num), index] = 1
  return output

def one_hot_if_needed(label, vocab_size=vocab_size):
  """Creates one-hot vectors to encode the label if it is not already one-hot.

  e.g. one_hot(label=2, vocab_size=5) = [0, 0, 1, 0, 0].
  e.g. one_hot(label=[0, 0, 1, 0, 0], vocab_size=5) = [0, 0, 1, 0, 0].
  """
  label = np.asarray(label)
  if len(label.shape) <= 1:
    label = one_hot(label, vocab_size)
  assert len(label.shape) == 2
  return label

def sample(sess, noise, label, truncation=1., batch_size=8,
           vocab_size=vocab_size):
  """Generate an image by sampling the generative model."""
  noise = np.asarray(noise)
  label = np.asarray(label)
  num = noise.shape[0]
  if len(label.shape) == 0:
    label = np.asarray([label] * num)
  label = one_hot_if_needed(label, vocab_size)
  ims = []
  for batch_start in range(0, num, batch_size):
    s = slice(batch_start, min(num, batch_start + batch_size))
    feed_dict = {input_z: noise[s], input_y: label[s], input_trunc: truncation}
    ims.append(sess.run(output, feed_dict=feed_dict))
  ims = np.concatenate(ims, axis=0)
  assert ims.shape[0] == num
  ims = np.clip(((ims + 1) / 2.0) * 256, 0, 255)
  ims = np.uint8(ims)
  return ims

def interpolate(A, B, num_interps):
  """Linearly interpolate between vectors A and B.
  
  e.g. interpolation = a*A + (1 - a)*B for a in [0, ..., 1]
  """
  alphas = np.linspace(0, 1, num_interps)
  return np.array([(1-a)*A + a*B for a in alphas])

def imgrid(imarray, cols=5, pad=1):
  """Function to display images in a grid."""
  pad = int(pad)
  assert pad >= 0
  cols = int(cols)
  assert cols >= 1
  N, H, W, C = imarray.shape
  rows = N // cols + int(N % cols != 0)
  batch_pad = rows * cols - N
  assert batch_pad >= 0
  post_pad = [batch_pad, pad, pad, 0]
  pad_arg = [[0, p] for p in post_pad]
  imarray = np.pad(imarray, pad_arg, 'constant', constant_values=255)
  H += pad
  W += pad
  grid = (imarray
          .reshape(rows, cols, H, W, C)
          .transpose(0, 2, 1, 3, 4)
          .reshape(rows*H, cols*W, C))
  if pad:
    grid = grid[:-pad, :-pad]
  return grid

# URLs are taken from ImageNet.
real_image_urls = ['http://static.flickr.com/27/65922179_fe6774cccf.jpg', 'https://farm3.static.flickr.com/2223/1589046073_5420d1ca1d.jpg', 'http://farm3.static.flickr.com/2327/2047231704_fcc0270193.jpg', 'http://www.fungimag.com/summer-08-articles/shiitakesepia2_sm.jpg', 'http://static.flickr.com/39/78374930_f0260bb501.jpg', 'http://farm2.static.flickr.com/1104/526094174_f627d7d2b9.jpg', 'http://farm1.static.flickr.com/253/451338953_20c68143f6.jpg', 'http://farm4.static.flickr.com/3149/2592226069_2291eddfd6_o.jpg', 'https://static.flickr.com/184/424890137_b09ac2c9c4.jpg']

#@title Create the quiz session.
reward = []
sess = tf.train.SingularMonitoredSession()

counter=0

Next: let's play the game!

In [ ]:
#@title Quiz: Is this image real or fake? {run: "auto"}
#@markdown You will score +1 point if you get the answer right and -1 point if you are wrong. How many can you get right?
def count_down(time_in_seconds):
  for i in range(time_in_seconds):
    time.sleep(1)
    # print("Only {} seconds left!".format(time_in_seconds - i))



def ask_question(inputs=None):
  # Randomly choose to generate a real of fake image.
  p = np.random.uniform(low=0.0, high=1.0)
  if p > 0.3:  # Bias towards showing more fake samples.
    label = "Fake"
    # Generate a fake image.
    z = truncated_z_sample(batch_size=1, truncation=0.05)
    y = int(np.random.uniform(low=0, high=1000))
    image = sample(sess, z, y, truncation=0.5)[0]
  else:
    # Generate a real image.
    label = "Real"
    url = np.random.choice(real_image_urls)
    image = io.imread(url)

  # Keep track of the number of correct predictions.
  grid = widgets.Grid(2, 2)

  with grid.output_to(0, 0):
    plt.plot(reward, 'b*')
    plt.xlabel("Images shown across time.")
    plt.ylabel("Score.")
    plt.ylim((-1.1, 1.1))

  with grid.output_to(0, 1):
    fig=plt.imshow(image)
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.show()
    

  with grid.output_to(1, 1):
    real_or_fake_button = ipywidgets.RadioButtons(
      options=['Real', 'Fake'],
      value='Fake',
      description='This image is\n',
      disabled=False)
    display(real_or_fake_button)
  submit_button = ipywidgets.Button(description='Submit')
  display(submit_button)
  next_button = ipywidgets.Button(description='Next')

  def check_prediction(inputs=None):
    print("Checking your answer...")
    prediction = real_or_fake_button.value
    if prediction == label:
      reward.append(1)
      print(emoji.emojize('Yes! :thumbs_up: This image was {}'.format(label)))
    else:
      print(emoji.emojize('Oh no :astonished_face:, this was a {} image.'.format(label)))
      reward.append(-1)
    print("Try another one.")
    submit_button.disabled = True
    display(next_button)

  def run_again(inputs=None):
    with IPython.utils.io.capture_output(display=False):
      IPython.display.clear_output()
      ask_question()

  submit_button.on_click(check_prediction)
  next_button.on_click(run_again)

if counter < 10:
  ask_question()
else:
  print("You have reached the end of the quiz.")


## These images are created by an AI system called a Generative Model.

The AI that creates these realistic looking images is a special type of AI, called a generative model, because it generates “something”, in this case images.

<center>
<img src="https://storage.googleapis.com/dm-educational/assets/generative/artist.jpg" alt="drawing" height="220"/>
</center>







# How does this work? 




This particular generative model is called a Generative Adversarial Network because it is made up of two components that compete against each other: The Artist and the Detective.

<center>
<img src="https://storage.googleapis.com/dm-educational/assets/generative/artist_and_detective.png" alt="drawing" height="220"/>
</center>


The Artist tries to create images that look realistic. 

<center>
<img src="https://storage.googleapis.com/dm-educational/assets/generative/artist_gen.png" alt="drawing" height="220"/>
</center>


The Detective is given some real images and some images from the Artist and has to predict which ones are real or fake, just like **you** did in the quiz above.


<center>

<img src="https://storage.googleapis.com/dm-educational/assets/generative/this_looks_real.png" alt="drawing" height="240"/><img src="https://storage.googleapis.com/dm-educational/assets/generative/this_looks_fake.png" alt="drawing" height="240"/>

</center>

At the start of the game neither the Artist nor the Detective are very good. After a few rounds the Detective gets better at detecting which images from the Artist are fake. For the Artist to win the game, they have to get better too, creating more and more realistic looking images as the game goes on.

## Why are generative models important?

Throughout the centuries humans have been faced with many challenges from *\"How do we get across this river?\"* to *\"How can we cure or prevent this disease?\"*

<figure class="image">
<center>
  <img src="https://letsgetsciencey.com/wp-content/uploads/2019/03/wright_jane.jpg" alt="Dr Jane">
  <figcaption> <b>Dr. Jane C. Wright </b> pioneered research into chemotherapy treatments for cancer. </figcaption>
</center>
</figure>
 
 
Humans came up with solutions to these problems, not only by looking in text books and asking others what the answer might be, but through **creative problem solving**. Scientists and engineers worked together to imagine many possible solutions and tried them until they found a solution that worked.
 
For AI to help us solve the current and future challenges that humanity will face, it too **needs to be creative** and needs to be able to imagine new things. We will now explore this below.









# Generative models are able to exhibit some level of imagination.
 
Firstly, if a generative model creates an image of a dog, it is probably not a dog that exists in the real world, it is one that it has imagined.
 
Generative models can go one step further than just imagining a dog that does not exist, they can also imagine objects categories that do not exist, by combining object categories that do exist.

<center>
<img src="https://storage.googleapis.com/dm-educational/assets/generative/imagine_unicorn.png" alt="drawing" height="220"/>
</center>
 

## See for yourself!
In the cell below you can select two categories: **`category_A`** and **`category_B`** from the drop down menu and look at how the generative model combines these; imaging images from the selected object categories.

In [ ]:
#@title Generative models imagining new object categories by merging two existing categories.

# Taken from https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/biggan_generation_with_tf_hub.ipynb
# with minor modificiations.

num_samples = 1
num_interps = 5
truncation = 0.2
noise_seed_A = 0
category_A = "0) tench, Tinca tinca" #@param ["0) tench, Tinca tinca", "1) goldfish, Carassius auratus", "2) great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias", "3) tiger shark, Galeocerdo cuvieri", "4) hammerhead, hammerhead shark", "5) electric ray, crampfish, numbfish, torpedo", "6) stingray", "7) cock", "8) hen", "9) ostrich, Struthio camelus", "10) brambling, Fringilla montifringilla", "11) goldfinch, Carduelis carduelis", "12) house finch, linnet, Carpodacus mexicanus", "13) junco, snowbird", "14) indigo bunting, indigo finch, indigo bird, Passerina cyanea", "15) robin, American robin, Turdus migratorius", "16) bulbul", "17) jay", "18) magpie", "19) chickadee", "20) water ouzel, dipper", "21) kite", "22) bald eagle, American eagle, Haliaeetus leucocephalus", "23) vulture", "24) great grey owl, great gray owl, Strix nebulosa", "25) European fire salamander, Salamandra salamandra", "26) common newt, Triturus vulgaris", "27) eft", "28) spotted salamander, Ambystoma maculatum", "29) axolotl, mud puppy, Ambystoma mexicanum", "30) bullfrog, Rana catesbeiana", "31) tree frog, tree-frog", "32) tailed frog, bell toad, ribbed toad, tailed toad, Ascaphus trui", "33) loggerhead, loggerhead turtle, Caretta caretta", "34) leatherback turtle, leatherback, leathery turtle, Dermochelys coriacea", "35) mud turtle", "36) terrapin", "37) box turtle, box tortoise", "38) banded gecko", "39) common iguana, iguana, Iguana iguana", "40) American chameleon, anole, Anolis carolinensis", "41) whiptail, whiptail lizard", "42) agama", "43) frilled lizard, Chlamydosaurus kingi", "44) alligator lizard", "45) Gila monster, Heloderma suspectum", "46) green lizard, Lacerta viridis", "47) African chameleon, Chamaeleo chamaeleon", "48) Komodo dragon, Komodo lizard, dragon lizard, giant lizard, Varanus komodoensis", "49) African crocodile, Nile crocodile, Crocodylus niloticus", "50) American alligator, Alligator mississipiensis", "51) triceratops", "52) thunder snake, worm snake, Carphophis amoenus", "53) ringneck snake, ring-necked snake, ring snake", "54) hognose snake, puff adder, sand viper", "55) green snake, grass snake", "56) king snake, kingsnake", "57) garter snake, grass snake", "58) water snake", "59) vine snake", "60) night snake, Hypsiglena torquata", "61) boa constrictor, Constrictor constrictor", "62) rock python, rock snake, Python sebae", "63) Indian cobra, Naja naja", "64) green mamba", "65) sea snake", "66) horned viper, cerastes, sand viper, horned asp, Cerastes cornutus", "67) diamondback, diamondback rattlesnake, Crotalus adamanteus", "68) sidewinder, horned rattlesnake, Crotalus cerastes", "69) trilobite", "70) harvestman, daddy longlegs, Phalangium opilio", "71) scorpion", "72) black and gold garden spider, Argiope aurantia", "73) barn spider, Araneus cavaticus", "74) garden spider, Aranea diademata", "75) black widow, Latrodectus mactans", "76) tarantula", "77) wolf spider, hunting spider", "78) tick", "79) centipede", "80) black grouse", "81) ptarmigan", "82) ruffed grouse, partridge, Bonasa umbellus", "83) prairie chicken, prairie grouse, prairie fowl", "84) peacock", "85) quail", "86) partridge", "87) African grey, African gray, Psittacus erithacus", "88) macaw", "89) sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita", "90) lorikeet", "91) coucal", "92) bee eater", "93) hornbill", "94) hummingbird", "95) jacamar", "96) toucan", "97) drake", "98) red-breasted merganser, Mergus serrator", "99) goose", "100) black swan, Cygnus atratus", "101) tusker", "102) echidna, spiny anteater, anteater", "103) platypus, duckbill, duckbilled platypus, duck-billed platypus, Ornithorhynchus anatinus", "104) wallaby, brush kangaroo", "105) koala, koala bear, kangaroo bear, native bear, Phascolarctos cinereus", "106) wombat", "108) sea anemone, anemone", "109) brain coral", "110) flatworm, platyhelminth", "111) nematode, nematode worm, roundworm", "112) conch", "113) snail", "114) slug", "115) sea slug, nudibranch", "118) Dungeness crab, Cancer magister", "119) rock crab, Cancer irroratus", "120) fiddler crab", "121) king crab, Alaska crab, Alaskan king crab, Alaska king crab, Paralithodes camtschatica", "122) American lobster, Northern lobster, Maine lobster, Homarus americanus", "123) spiny lobster, langouste, rock lobster, crawfish, crayfish, sea crawfish", "124) crayfish, crawfish, crawdad, crawdaddy", "125) hermit crab", "126) isopod", "127) white stork, Ciconia ciconia", "128) black stork, Ciconia nigra", "129) spoonbill", "130) flamingo", "131) little blue heron, Egretta caerulea", "132) American egret, great white heron, Egretta albus", "133) bittern", "517) crane", "135) limpkin, Aramus pictus", "136) European gallinule, Porphyrio porphyrio", "137) American coot, marsh hen, mud hen, water hen, Fulica americana", "138) bustard", "139) ruddy turnstone, Arenaria interpres", "141) redshank, Tringa totanus", "142) dowitcher", "143) oystercatcher, oyster catcher", "144) pelican", "145) king penguin, Aptenodytes patagonica", "146) albatross, mollymawk", "147) grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus", "148) killer whale, killer, orca, grampus, sea wolf, Orcinus orca", "149) dugong, Dugong dugon", "150) sea lion", "151) Chihuahua", "152) Japanese spaniel", "153) Maltese dog, Maltese terrier, Maltese", "154) Pekinese, Pekingese, Peke", "155) Shih-Tzu", "156) Blenheim spaniel", "157) papillon", "158) toy terrier", "159) Rhodesian ridgeback", "160) Afghan hound, Afghan", "161) basset, basset hound", "162) beagle", "163) bloodhound, sleuthhound", "164) bluetick", "165) black-and-tan coonhound", "166) Walker hound, Walker foxhound", "167) English foxhound", "168) redbone", "169) borzoi, Russian wolfhound", "170) Irish wolfhound", "171) Italian greyhound", "172) whippet", "173) Ibizan hound, Ibizan Podenco", "174) Norwegian elkhound, elkhound", "175) otterhound, otter hound", "176) Saluki, gazelle hound", "177) Scottish deerhound, deerhound", "178) Weimaraner", "179) Staffordshire bullterrier, Staffordshire bull terrier", "180) American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier", "181) Bedlington terrier", "182) Border terrier", "183) Kerry blue terrier", "184) Irish terrier", "185) Norfolk terrier", "186) Norwich terrier", "187) Yorkshire terrier", "188) wire-haired fox terrier", "189) Lakeland terrier", "190) Sealyham terrier, Sealyham", "191) Airedale, Airedale terrier", "192) cairn, cairn terrier", "193) Australian terrier", "194) Dandie Dinmont, Dandie Dinmont terrier", "195) Boston bull, Boston terrier", "196) miniature schnauzer", "197) giant schnauzer", "198) standard schnauzer", "199) Scotch terrier, Scottish terrier, Scottie", "200) Tibetan terrier, chrysanthemum dog", "201) silky terrier, Sydney silky", "202) soft-coated wheaten terrier", "203) West Highland white terrier", "204) Lhasa, Lhasa apso", "205) flat-coated retriever", "206) curly-coated retriever", "207) golden retriever", "208) Labrador retriever", "209) Chesapeake Bay retriever", "210) German short-haired pointer", "211) vizsla, Hungarian pointer", "212) English setter", "213) Irish setter, red setter", "214) Gordon setter", "215) Brittany spaniel", "216) clumber, clumber spaniel", "217) English springer, English springer spaniel", "218) Welsh springer spaniel", "219) cocker spaniel, English cocker spaniel, cocker", "220) Sussex spaniel", "221) Irish water spaniel", "222) kuvasz", "223) schipperke", "224) groenendael", "225) malinois", "226) briard", "227) kelpie", "228) komondor", "229) Old English sheepdog, bobtail", "230) Shetland sheepdog, Shetland sheep dog, Shetland", "231) collie", "232) Border collie", "233) Bouvier des Flandres, Bouviers des Flandres", "234) Rottweiler", "235) German shepherd, German shepherd dog, German police dog, alsatian", "236) Doberman, Doberman pinscher", "237) miniature pinscher", "238) Greater Swiss Mountain dog", "239) Bernese mountain dog", "240) Appenzeller", "241) EntleBucher", "242) boxer", "243) bull mastiff", "244) Tibetan mastiff", "245) French bulldog", "246) Great Dane", "247) Saint Bernard, St Bernard", "248) Eskimo dog, husky", "249) malamute, malemute, Alaskan malamute", "250) Siberian husky", "251) dalmatian, coach dog, carriage dog", "252) affenpinscher, monkey pinscher, monkey dog", "253) basenji", "254) pug, pug-dog", "255) Leonberg", "256) Newfoundland, Newfoundland dog", "257) Great Pyrenees", "258) Samoyed, Samoyede", "259) Pomeranian", "260) chow, chow chow", "261) keeshond", "262) Brabancon griffon", "263) Pembroke, Pembroke Welsh corgi", "264) Cardigan, Cardigan Welsh corgi", "265) toy poodle", "266) miniature poodle", "267) standard poodle", "268) Mexican hairless", "269) timber wolf, grey wolf, gray wolf, Canis lupus", "270) white wolf, Arctic wolf, Canis lupus tundrarum", "271) red wolf, maned wolf, Canis rufus, Canis niger", "272) coyote, prairie wolf, brush wolf, Canis latrans", "273) dingo, warrigal, warragal, Canis dingo", "274) dhole, Cuon alpinus", "275) African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus", "276) hyena, hyaena", "277) red fox, Vulpes vulpes", "278) kit fox, Vulpes macrotis", "279) Arctic fox, white fox, Alopex lagopus", "280) grey fox, gray fox, Urocyon cinereoargenteus", "281) tabby, tabby cat", "282) tiger cat", "283) Persian cat", "284) Siamese cat, Siamese", "285) Egyptian cat", "286) cougar, puma, catamount, mountain lion, painter, panther, Felis concolor", "287) lynx, catamount", "288) leopard, Panthera pardus", "289) snow leopard, ounce, Panthera uncia", "290) jaguar, panther, Panthera onca, Felis onca", "291) lion, king of beasts, Panthera leo", "292) tiger, Panthera tigris", "293) cheetah, chetah, Acinonyx jubatus", "294) brown bear, bruin, Ursus arctos", "295) American black bear, black bear, Ursus americanus, Euarctos americanus", "296) ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus", "297) sloth bear, Melursus ursinus, Ursus ursinus", "298) mongoose", "299) meerkat, mierkat", "300) tiger beetle", "301) ladybug, ladybeetle, lady beetle, ladybird, ladybird beetle", "302) ground beetle, carabid beetle", "303) long-horned beetle, longicorn, longicorn beetle", "304) leaf beetle, chrysomelid", "305) dung beetle", "306) rhinoceros beetle", "307) weevil", "308) fly", "309) bee", "310) ant, emmet, pismire", "311) grasshopper, hopper", "312) cricket", "313) walking stick, walkingstick, stick insect", "314) cockroach, roach", "315) mantis, mantid", "316) cicada, cicala", "317) leafhopper", "318) lacewing, lacewing fly", "319) dragonfly, darning needle, devil's darning needle, sewing needle, snake feeder, snake doctor, mosquito hawk, skeeter hawk", "320) damselfly", "321) admiral", "322) ringlet, ringlet butterfly", "323) monarch, monarch butterfly, milkweed butterfly, Danaus plexippus", "324) cabbage butterfly", "325) sulphur butterfly, sulfur butterfly", "326) lycaenid, lycaenid butterfly", "327) starfish, sea star", "328) sea urchin", "329) sea cucumber, holothurian", "330) wood rabbit, cottontail, cottontail rabbit", "331) hare", "332) Angora, Angora rabbit", "333) hamster", "334) porcupine, hedgehog", "335) fox squirrel, eastern fox squirrel, Sciurus niger", "336) marmot", "337) beaver", "338) guinea pig, Cavia cobaya", "339) sorrel", "340) zebra", "341) hog, pig, grunter, squealer, Sus scrofa", "342) wild boar, boar, Sus scrofa", "343) warthog", "344) hippopotamus, hippo, river horse, Hippopotamus amphibius", "345) ox", "346) water buffalo, water ox, Asiatic buffalo, Bubalus bubalis", "347) bison", "348) ram, tup", "349) bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis canadensis", "350) ibex, Capra ibex", "351) hartebeest", "352) impala, Aepyceros melampus", "353) gazelle", "354) Arabian camel, dromedary, Camelus dromedarius", "355) llama", "356) weasel", "357) mink", "358) polecat, fitch, foulmart, foumart, Mustela putorius", "359) black-footed ferret, ferret, Mustela nigripes", "360) otter", "361) skunk, polecat, wood pussy", "362) badger", "363) armadillo", "364) three-toed sloth, ai, Bradypus tridactylus", "365) orangutan, orang, orangutang, Pongo pygmaeus", "366) gorilla, Gorilla gorilla", "367) chimpanzee, chimp, Pan troglodytes", "368) gibbon, Hylobates lar", "369) siamang, Hylobates syndactylus, Symphalangus syndactylus", "370) guenon, guenon monkey", "371) patas, hussar monkey, Erythrocebus patas", "372) baboon", "373) macaque", "374) langur", "375) colobus, colobus monkey", "376) proboscis monkey, Nasalis larvatus", "377) marmoset", "378) capuchin, ringtail, Cebus capucinus", "379) howler monkey, howler", "380) titi, titi monkey", "381) spider monkey, Ateles geoffroyi", "382) squirrel monkey, Saimiri sciureus", "383) Madagascar cat, ring-tailed lemur, Lemur catta", "384) indri, indris, Indri indri, Indri brevicaudatus", "385) Indian elephant, Elephas maximus", "386) African elephant, Loxodonta africana", "387) lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens", "388) giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca", "389) barracouta, snoek", "390) eel", "391) coho, cohoe, coho salmon, blue jack, silver salmon, Oncorhynchus kisutch", "392) rock beauty, Holocanthus tricolor", "393) anemone fish", "394) sturgeon", "395) gar, garfish, garpike, billfish, Lepisosteus osseus", "396) lionfish", "398) abacus", "399) abaya", "400) academic gown, academic robe, judge's robe", "401) accordion, piano accordion, squeeze box", "402) acoustic guitar", "403) aircraft carrier, carrier, flattop, attack aircraft carrier", "404) airliner", "405) airship, dirigible", "406) altar", "407) ambulance", "408) amphibian, amphibious vehicle", "409) analog clock", "410) apiary, bee house", "411) apron", "412) ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin", "413) assault rifle, assault gun", "414) backpack, back pack, knapsack, packsack, rucksack, haversack", "415) bakery, bakeshop, bakehouse", "416) balance beam, beam", "417) balloon", "418) ballpoint, ballpoint pen, ballpen, Biro", "419) Band Aid", "420) banjo", "421) bannister, banister, balustrade, balusters, handrail", "422) barbell", "423) barber chair", "424) barbershop", "425) barn", "426) barometer", "427) barrel, cask", "428) barrow, garden cart, lawn cart, wheelbarrow", "429) baseball", "430) basketball", "431) bassinet", "432) bassoon", "433) bathing cap, swimming cap", "434) bath towel", "435) bathtub, bathing tub, bath, tub", "436) beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon", "437) beacon, lighthouse, beacon light, pharos", "438) beaker", "439) bearskin, busby, shako", "440) beer bottle", "441) beer glass", "442) bell cote, bell cot", "443) bib", "444) bicycle-built-for-two, tandem bicycle, tandem", "445) bikini, two-piece", "446) binder, ring-binder", "447) binoculars, field glasses, opera glasses", "448) birdhouse", "449) boathouse", "450) bobsled, bobsleigh, bob", "451) bolo tie, bolo, bola tie, bola", "452) bonnet, poke bonnet", "453) bookcase", "454) bookshop, bookstore, bookstall", "455) bottlecap", "456) bow", "457) bow tie, bow-tie, bowtie", "458) brass, memorial tablet, plaque", "459) brassiere, bra, bandeau", "460) breakwater, groin, groyne, mole, bulwark, seawall, jetty", "461) breastplate, aegis, egis", "462) broom", "463) bucket, pail", "464) buckle", "465) bulletproof vest", "466) bullet train, bullet", "467) butcher shop, meat market", "469) caldron, cauldron", "470) candle, taper, wax light", "471) cannon", "472) canoe", "473) can opener, tin opener", "474) cardigan", "475) car mirror", "476) carousel, carrousel, merry-go-round, roundabout, whirligig", "477) carpenter's kit, tool kit", "478) carton", "479) car wheel", "480) cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM", "481) cassette", "482) cassette player", "483) castle", "484) catamaran", "485) CD player", "486) cello, violoncello", "487) cellular telephone, cellular phone, cellphone, cell, mobile phone", "488) chain", "489) chainlink fence", "490) chain mail, ring mail, mail, chain armor, chain armour, ring armor, ring armour", "491) chain saw, chainsaw", "492) chest", "493) chiffonier, commode", "494) chime, bell, gong", "495) china cabinet, china closet", "496) Christmas stocking", "497) church, church building", "498) cinema, movie theater, movie theatre, movie house, picture palace", "499) cleaver, meat cleaver, chopper", "500) cliff dwelling", "501) cloak", "502) clog, geta, patten, sabot", "503) cocktail shaker", "504) coffee mug", "505) coffeepot", "506) coil, spiral, volute, whorl, helix", "507) combination lock", "508) computer keyboard, keypad", "509) confectionery, confectionary, candy store", "510) container ship, containership, container vessel", "511) convertible", "512) corkscrew, bottle screw", "513) cornet, horn, trumpet, trump", "514) cowboy boot", "515) cowboy hat, ten-gallon hat", "516) cradle", "517) crane", "518) crash helmet", "519) crate", "520) crib, cot", "521) Crock Pot", "522) croquet ball", "523) crutch", "524) cuirass", "525) dam, dike, dyke", "526) desk", "527) desktop computer", "528) dial telephone, dial phone", "529) diaper, nappy, napkin", "530) digital clock", "531) digital watch", "532) dining table, board", "533) dishrag, dishcloth", "534) dishwasher, dish washer, dishwashing machine", "535) disk brake, disc brake", "536) dock, dockage, docking facility", "537) dogsled, dog sled, dog sleigh", "538) dome", "539) doormat, welcome mat", "540) drilling platform, offshore rig", "541) drum, membranophone, tympan", "542) drumstick", "543) dumbbell", "544) Dutch oven", "545) electric fan, blower", "546) electric guitar", "547) electric locomotive", "548) entertainment center", "549) envelope", "550) espresso maker", "551) face powder", "552) feather boa, boa", "553) file, file cabinet, filing cabinet", "554) fireboat", "555) fire engine, fire truck", "556) fire screen, fireguard", "557) flagpole, flagstaff", "558) flute, transverse flute", "559) folding chair", "560) football helmet", "561) forklift", "562) fountain", "563) fountain pen", "564) four-poster", "565) freight car", "566) French horn, horn", "567) frying pan, frypan, skillet", "568) fur coat", "569) garbage truck, dustcart", "570) gasmask, respirator, gas helmet", "571) gas pump, gasoline pump, petrol pump, island dispenser", "572) goblet", "573) go-kart", "574) golf ball", "575) golfcart, golf cart", "576) gondola", "577) gong, tam-tam", "578) gown", "579) grand piano, grand", "580) greenhouse, nursery, glasshouse", "581) grille, radiator grille", "582) grocery store, grocery, food market, market", "583) guillotine", "584) hair slide", "585) hair spray", "586) half track", "587) hammer", "588) hamper", "589) hand blower, blow dryer, blow drier, hair dryer, hair drier", "590) hand-held computer, hand-held microcomputer", "591) handkerchief, hankie, hanky, hankey", "592) hard disc, hard disk, fixed disk", "593) harmonica, mouth organ, harp, mouth harp", "594) harp", "595) harvester, reaper", "596) hatchet", "597) holster", "598) home theater, home theatre", "599) honeycomb", "600) hook, claw", "601) hoopskirt, crinoline", "602) horizontal bar, high bar", "603) horse cart, horse-cart", "604) hourglass", "605) iPod", "606) iron, smoothing iron", "607) jack-o'-lantern", "608) jean, blue jean, denim", "609) jeep, landrover", "610) jersey, T-shirt, tee shirt", "611) jigsaw puzzle", "612) jinrikisha, ricksha, rickshaw", "613) joystick", "614) kimono", "615) knee pad", "616) knot", "617) lab coat, laboratory coat", "618) ladle", "619) lampshade, lamp shade", "620) laptop, laptop computer", "621) lawn mower, mower", "622) lens cap, lens cover", "623) letter opener, paper knife, paperknife", "624) library", "625) lifeboat", "626) lighter, light, igniter, ignitor", "627) limousine, limo", "628) liner, ocean liner", "629) lipstick, lip rouge", "630) Loafer", "631) lotion", "632) loudspeaker, speaker, speaker unit, loudspeaker system, speaker system", "633) loupe, jeweler's loupe", "634) lumbermill, sawmill", "635) magnetic compass", "636) mailbag, postbag", "637) mailbox, letter box", "638) maillot", "639) maillot, tank suit", "640) manhole cover", "641) maraca", "642) marimba, xylophone", "643) mask", "644) matchstick", "645) maypole", "646) maze, labyrinth", "647) measuring cup", "648) medicine chest, medicine cabinet", "649) megalith, megalithic structure", "650) microphone, mike", "651) microwave, microwave oven", "652) military uniform", "653) milk can", "654) minibus", "655) miniskirt, mini", "656) minivan", "657) missile", "658) mitten", "659) mixing bowl", "660) mobile home, manufactured home", "661) Model T", "662) modem", "663) monastery", "664) monitor", "665) moped", "666) mortar", "667) mortarboard", "668) mosque", "669) mosquito net", "670) motor scooter, scooter", "671) mountain bike, all-terrain bike, off-roader", "672) mountain tent", "673) mouse, computer mouse", "674) mousetrap", "675) moving van", "676) muzzle", "677) nail", "678) neck brace", "679) necklace", "680) nipple", "681) notebook, notebook computer", "682) obelisk", "683) oboe, hautboy, hautbois", "684) ocarina, sweet potato", "685) odometer, hodometer, mileometer, milometer", "686) oil filter", "687) organ, pipe organ", "688) oscilloscope, scope, cathode-ray oscilloscope, CRO", "689) overskirt", "690) oxcart", "691) oxygen mask", "692) packet", "693) paddle, boat paddle", "694) paddlewheel, paddle wheel", "695) padlock", "696) paintbrush", "697) pajama, pyjama, pj's, jammies", "698) palace", "699) panpipe, pandean pipe, syrinx", "700) paper towel", "701) parachute, chute", "702) parallel bars, bars", "703) park bench", "704) parking meter", "705) passenger car, coach, carriage", "706) patio, terrace", "707) pay-phone, pay-station", "708) pedestal, plinth, footstall", "709) pencil box, pencil case", "710) pencil sharpener", "711) perfume, essence", "712) Petri dish", "713) photocopier", "714) pick, plectrum, plectron", "715) pickelhaube", "716) picket fence, paling", "717) pickup, pickup truck", "718) pier", "719) piggy bank, penny bank", "720) pill bottle", "721) pillow", "722) ping-pong ball", "723) pinwheel", "724) pirate, pirate ship", "725) pitcher, ewer", "726) plane, carpenter's plane, woodworking plane", "727) planetarium", "728) plastic bag", "729) plate rack", "730) plow, plough", "731) plunger, plumber's helper", "732) Polaroid camera, Polaroid Land camera", "733) pole", "734) police van, police wagon, paddy wagon, patrol wagon, wagon, black Maria", "735) poncho", "736) pool table, billiard table, snooker table", "737) pop bottle, soda bottle", "738) pot, flowerpot", "739) potter's wheel", "740) power drill", "741) prayer rug, prayer mat", "742) printer", "743) prison, prison house", "744) projectile, missile", "745) projector", "746) puck, hockey puck", "747) punching bag, punch bag, punching ball, punchball", "748) purse", "749) quill, quill pen", "750) quilt, comforter, comfort, puff", "751) racer, race car, racing car", "752) racket, racquet", "753) radiator", "754) radio, wireless", "755) radio telescope, radio reflector", "756) rain barrel", "757) recreational vehicle, RV, R.V.", "758) reel", "759) reflex camera", "760) refrigerator, icebox", "761) remote control, remote", "762) restaurant, eating house, eating place, eatery", "763) revolver, six-gun, six-shooter", "764) rifle", "765) rocking chair, rocker", "766) rotisserie", "767) rubber eraser, rubber, pencil eraser", "768) rugby ball", "769) rule, ruler", "770) running shoe", "771) safe", "772) safety pin", "773) saltshaker, salt shaker", "774) sandal", "775) sarong", "776) sax, saxophone", "777) scabbard", "778) scale, weighing machine", "779) school bus", "780) schooner", "781) scoreboard", "782) screen, CRT screen", "783) screw", "784) screwdriver", "785) seat belt, seatbelt", "786) sewing machine", "787) shield, buckler", "788) shoe shop, shoe-shop, shoe store", "789) shoji", "790) shopping basket", "791) shopping cart", "792) shovel", "793) shower cap", "794) shower curtain", "795) ski", "796) ski mask", "797) sleeping bag", "798) slide rule, slipstick", "799) sliding door", "800) slot, one-armed bandit", "801) snorkel", "802) snowmobile", "803) snowplow, snowplough", "804) soap dispenser", "805) soccer ball", "806) sock", "807) solar dish, solar collector, solar furnace", "808) sombrero", "809) soup bowl", "810) space bar", "811) space heater", "812) space shuttle", "813) spatula", "814) speedboat", "815) spider web, spider's web", "816) spindle", "817) sports car, sport car", "818) spotlight, spot", "819) stage", "820) steam locomotive", "821) steel arch bridge", "822) steel drum", "823) stethoscope", "824) stole", "825) stone wall", "826) stopwatch, stop watch", "827) stove", "828) strainer", "829) streetcar, tram, tramcar, trolley, trolley car", "830) stretcher", "831) studio couch, day bed", "832) stupa, tope", "833) submarine, pigboat, sub, U-boat", "834) suit, suit of clothes", "835) sundial", "836) sunglass", "837) sunglasses, dark glasses, shades", "838) sunscreen, sunblock, sun blocker", "839) suspension bridge", "840) swab, swob, mop", "841) sweatshirt", "842) swimming trunks, bathing trunks", "843) swing", "844) switch, electric switch, electrical switch", "845) syringe", "846) table lamp", "847) tank, army tank, armored combat vehicle, armoured combat vehicle", "848) tape player", "849) teapot", "850) teddy, teddy bear", "851) television, television system", "852) tennis ball", "853) thatch, thatched roof", "854) theater curtain, theatre curtain", "855) thimble", "856) thresher, thrasher, threshing machine", "857) throne", "858) tile roof", "859) toaster", "860) tobacco shop, tobacconist shop, tobacconist", "861) toilet seat", "862) torch", "863) totem pole", "864) tow truck, tow car, wrecker", "865) toyshop", "866) tractor", "867) trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi", "868) tray", "869) trench coat", "870) tricycle, trike, velocipede", "871) trimaran", "872) tripod", "873) triumphal arch", "874) trolleybus, trolley coach, trackless trolley", "875) trombone", "876) tub, vat", "877) turnstile", "878) typewriter keyboard", "879) umbrella", "880) unicycle, monocycle", "881) upright, upright piano", "882) vacuum, vacuum cleaner", "883) vase", "884) vault", "885) velvet", "886) vending machine", "887) vestment", "888) viaduct", "889) violin, fiddle", "890) volleyball", "891) waffle iron", "892) wall clock", "893) wallet, billfold, notecase, pocketbook", "894) wardrobe, closet, press", "895) warplane, military plane", "896) washbasin, handbasin, washbowl, lavabo, wash-hand basin", "897) washer, automatic washer, washing machine", "898) water bottle", "899) water jug", "900) water tower", "901) whiskey jug", "902) whistle", "903) wig", "904) window screen", "905) window shade", "906) Windsor tie", "907) wine bottle", "908) wing", "909) wok", "910) wooden spoon", "911) wool, woolen, woollen", "912) worm fence, snake fence, snake-rail fence, Virginia fence", "913) wreck", "914) yawl", "915) yurt", "916) web site, website, internet site, site", "917) comic book", "918) crossword puzzle, crossword", "919) street sign", "920) traffic light, traffic signal, stoplight", "921) book jacket, dust cover, dust jacket, dust wrapper", "922) menu", "923) plate", "924) guacamole", "925) consomme", "926) hot pot, hotpot", "927) trifle", "928) ice cream, icecream", "929) ice lolly, lolly, lollipop, popsicle", "930) French loaf", "931) bagel, beigel", "932) pretzel", "933) cheeseburger", "934) hotdog, hot dog, red hot", "935) mashed potato", "936) head cabbage", "937) broccoli", "938) cauliflower", "939) zucchini, courgette", "940) spaghetti squash", "941) acorn squash", "942) butternut squash", "943) cucumber, cuke", "944) artichoke, globe artichoke", "945) bell pepper", "946) cardoon", "947) mushroom", "948) Granny Smith", "949) strawberry", "950) orange", "951) lemon", "952) fig", "953) pineapple, ananas", "954) banana", "955) jackfruit, jak, jack", "956) custard apple", "957) pomegranate", "958) hay", "959) carbonara", "960) chocolate sauce, chocolate syrup", "961) dough", "962) meat loaf, meatloaf", "963) pizza, pizza pie", "964) potpie", "965) burrito", "966) red wine", "967) espresso", "968) cup", "969) eggnog", "970) alp", "971) bubble", "972) cliff, drop, drop-off", "973) coral reef", "974) geyser", "975) lakeside, lakeshore", "976) promontory, headland, head, foreland", "977) sandbar, sand bar", "978) seashore, coast, seacoast, sea-coast", "979) valley, vale", "980) volcano", "981) ballplayer, baseball player", "982) groom, bridegroom", "983) scuba diver", "984) rapeseed", "985) daisy", "986) yellow lady's slipper, yellow lady-slipper, Cypripedium calceolus, Cypripedium parviflorum", "987) corn", "988) acorn", "989) hip, rose hip, rosehip", "990) buckeye, horse chestnut, conker", "991) coral fungus", "992) agaric", "993) gyromitra", "994) stinkhorn, carrion fungus", "995) earthstar", "997) bolete", "998) ear, spike, capitulum", "999) toilet tissue, toilet paper, bathroom tissue"]

noise_seed_B = 0
category_B = "0) tench, Tinca tinca" #@param ["0) tench, Tinca tinca", "1) goldfish, Carassius auratus", "2) great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias", "3) tiger shark, Galeocerdo cuvieri", "4) hammerhead, hammerhead shark", "5) electric ray, crampfish, numbfish, torpedo", "6) stingray", "7) cock", "8) hen", "9) ostrich, Struthio camelus", "10) brambling, Fringilla montifringilla", "11) goldfinch, Carduelis carduelis", "12) house finch, linnet, Carpodacus mexicanus", "13) junco, snowbird", "14) indigo bunting, indigo finch, indigo bird, Passerina cyanea", "15) robin, American robin, Turdus migratorius", "16) bulbul", "17) jay", "18) magpie", "19) chickadee", "20) water ouzel, dipper", "21) kite", "22) bald eagle, American eagle, Haliaeetus leucocephalus", "23) vulture", "24) great grey owl, great gray owl, Strix nebulosa", "25) European fire salamander, Salamandra salamandra", "26) common newt, Triturus vulgaris", "27) eft", "28) spotted salamander, Ambystoma maculatum", "29) axolotl, mud puppy, Ambystoma mexicanum", "30) bullfrog, Rana catesbeiana", "31) tree frog, tree-frog", "32) tailed frog, bell toad, ribbed toad, tailed toad, Ascaphus trui", "33) loggerhead, loggerhead turtle, Caretta caretta", "34) leatherback turtle, leatherback, leathery turtle, Dermochelys coriacea", "35) mud turtle", "36) terrapin", "37) box turtle, box tortoise", "38) banded gecko", "39) common iguana, iguana, Iguana iguana", "40) American chameleon, anole, Anolis carolinensis", "41) whiptail, whiptail lizard", "42) agama", "43) frilled lizard, Chlamydosaurus kingi", "44) alligator lizard", "45) Gila monster, Heloderma suspectum", "46) green lizard, Lacerta viridis", "47) African chameleon, Chamaeleo chamaeleon", "48) Komodo dragon, Komodo lizard, dragon lizard, giant lizard, Varanus komodoensis", "49) African crocodile, Nile crocodile, Crocodylus niloticus", "50) American alligator, Alligator mississipiensis", "51) triceratops", "52) thunder snake, worm snake, Carphophis amoenus", "53) ringneck snake, ring-necked snake, ring snake", "54) hognose snake, puff adder, sand viper", "55) green snake, grass snake", "56) king snake, kingsnake", "57) garter snake, grass snake", "58) water snake", "59) vine snake", "60) night snake, Hypsiglena torquata", "61) boa constrictor, Constrictor constrictor", "62) rock python, rock snake, Python sebae", "63) Indian cobra, Naja naja", "64) green mamba", "65) sea snake", "66) horned viper, cerastes, sand viper, horned asp, Cerastes cornutus", "67) diamondback, diamondback rattlesnake, Crotalus adamanteus", "68) sidewinder, horned rattlesnake, Crotalus cerastes", "69) trilobite", "70) harvestman, daddy longlegs, Phalangium opilio", "71) scorpion", "72) black and gold garden spider, Argiope aurantia", "73) barn spider, Araneus cavaticus", "74) garden spider, Aranea diademata", "75) black widow, Latrodectus mactans", "76) tarantula", "77) wolf spider, hunting spider", "78) tick", "79) centipede", "80) black grouse", "81) ptarmigan", "82) ruffed grouse, partridge, Bonasa umbellus", "83) prairie chicken, prairie grouse, prairie fowl", "84) peacock", "85) quail", "86) partridge", "87) African grey, African gray, Psittacus erithacus", "88) macaw", "89) sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita", "90) lorikeet", "91) coucal", "92) bee eater", "93) hornbill", "94) hummingbird", "95) jacamar", "96) toucan", "97) drake", "98) red-breasted merganser, Mergus serrator", "99) goose", "100) black swan, Cygnus atratus", "101) tusker", "102) echidna, spiny anteater, anteater", "103) platypus, duckbill, duckbilled platypus, duck-billed platypus, Ornithorhynchus anatinus", "104) wallaby, brush kangaroo", "105) koala, koala bear, kangaroo bear, native bear, Phascolarctos cinereus", "106) wombat", "108) sea anemone, anemone", "109) brain coral", "110) flatworm, platyhelminth", "111) nematode, nematode worm, roundworm", "112) conch", "113) snail", "114) slug", "115) sea slug, nudibranch", "118) Dungeness crab, Cancer magister", "119) rock crab, Cancer irroratus", "120) fiddler crab", "121) king crab, Alaska crab, Alaskan king crab, Alaska king crab, Paralithodes camtschatica", "122) American lobster, Northern lobster, Maine lobster, Homarus americanus", "123) spiny lobster, langouste, rock lobster, crawfish, crayfish, sea crawfish", "124) crayfish, crawfish, crawdad, crawdaddy", "125) hermit crab", "126) isopod", "127) white stork, Ciconia ciconia", "128) black stork, Ciconia nigra", "129) spoonbill", "130) flamingo", "131) little blue heron, Egretta caerulea", "132) American egret, great white heron, Egretta albus", "133) bittern", "517) crane", "135) limpkin, Aramus pictus", "136) European gallinule, Porphyrio porphyrio", "137) American coot, marsh hen, mud hen, water hen, Fulica americana", "138) bustard", "139) ruddy turnstone, Arenaria interpres", "141) redshank, Tringa totanus", "142) dowitcher", "143) oystercatcher, oyster catcher", "144) pelican", "145) king penguin, Aptenodytes patagonica", "146) albatross, mollymawk", "147) grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus", "148) killer whale, killer, orca, grampus, sea wolf, Orcinus orca", "149) dugong, Dugong dugon", "150) sea lion", "151) Chihuahua", "152) Japanese spaniel", "153) Maltese dog, Maltese terrier, Maltese", "154) Pekinese, Pekingese, Peke", "155) Shih-Tzu", "156) Blenheim spaniel", "157) papillon", "158) toy terrier", "159) Rhodesian ridgeback", "160) Afghan hound, Afghan", "161) basset, basset hound", "162) beagle", "163) bloodhound, sleuthhound", "164) bluetick", "165) black-and-tan coonhound", "166) Walker hound, Walker foxhound", "167) English foxhound", "168) redbone", "169) borzoi, Russian wolfhound", "170) Irish wolfhound", "171) Italian greyhound", "172) whippet", "173) Ibizan hound, Ibizan Podenco", "174) Norwegian elkhound, elkhound", "175) otterhound, otter hound", "176) Saluki, gazelle hound", "177) Scottish deerhound, deerhound", "178) Weimaraner", "179) Staffordshire bullterrier, Staffordshire bull terrier", "180) American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier", "181) Bedlington terrier", "182) Border terrier", "183) Kerry blue terrier", "184) Irish terrier", "185) Norfolk terrier", "186) Norwich terrier", "187) Yorkshire terrier", "188) wire-haired fox terrier", "189) Lakeland terrier", "190) Sealyham terrier, Sealyham", "191) Airedale, Airedale terrier", "192) cairn, cairn terrier", "193) Australian terrier", "194) Dandie Dinmont, Dandie Dinmont terrier", "195) Boston bull, Boston terrier", "196) miniature schnauzer", "197) giant schnauzer", "198) standard schnauzer", "199) Scotch terrier, Scottish terrier, Scottie", "200) Tibetan terrier, chrysanthemum dog", "201) silky terrier, Sydney silky", "202) soft-coated wheaten terrier", "203) West Highland white terrier", "204) Lhasa, Lhasa apso", "205) flat-coated retriever", "206) curly-coated retriever", "207) golden retriever", "208) Labrador retriever", "209) Chesapeake Bay retriever", "210) German short-haired pointer", "211) vizsla, Hungarian pointer", "212) English setter", "213) Irish setter, red setter", "214) Gordon setter", "215) Brittany spaniel", "216) clumber, clumber spaniel", "217) English springer, English springer spaniel", "218) Welsh springer spaniel", "219) cocker spaniel, English cocker spaniel, cocker", "220) Sussex spaniel", "221) Irish water spaniel", "222) kuvasz", "223) schipperke", "224) groenendael", "225) malinois", "226) briard", "227) kelpie", "228) komondor", "229) Old English sheepdog, bobtail", "230) Shetland sheepdog, Shetland sheep dog, Shetland", "231) collie", "232) Border collie", "233) Bouvier des Flandres, Bouviers des Flandres", "234) Rottweiler", "235) German shepherd, German shepherd dog, German police dog, alsatian", "236) Doberman, Doberman pinscher", "237) miniature pinscher", "238) Greater Swiss Mountain dog", "239) Bernese mountain dog", "240) Appenzeller", "241) EntleBucher", "242) boxer", "243) bull mastiff", "244) Tibetan mastiff", "245) French bulldog", "246) Great Dane", "247) Saint Bernard, St Bernard", "248) Eskimo dog, husky", "249) malamute, malemute, Alaskan malamute", "250) Siberian husky", "251) dalmatian, coach dog, carriage dog", "252) affenpinscher, monkey pinscher, monkey dog", "253) basenji", "254) pug, pug-dog", "255) Leonberg", "256) Newfoundland, Newfoundland dog", "257) Great Pyrenees", "258) Samoyed, Samoyede", "259) Pomeranian", "260) chow, chow chow", "261) keeshond", "262) Brabancon griffon", "263) Pembroke, Pembroke Welsh corgi", "264) Cardigan, Cardigan Welsh corgi", "265) toy poodle", "266) miniature poodle", "267) standard poodle", "268) Mexican hairless", "269) timber wolf, grey wolf, gray wolf, Canis lupus", "270) white wolf, Arctic wolf, Canis lupus tundrarum", "271) red wolf, maned wolf, Canis rufus, Canis niger", "272) coyote, prairie wolf, brush wolf, Canis latrans", "273) dingo, warrigal, warragal, Canis dingo", "274) dhole, Cuon alpinus", "275) African hunting dog, hyena dog, Cape hunting dog, Lycaon pictus", "276) hyena, hyaena", "277) red fox, Vulpes vulpes", "278) kit fox, Vulpes macrotis", "279) Arctic fox, white fox, Alopex lagopus", "280) grey fox, gray fox, Urocyon cinereoargenteus", "281) tabby, tabby cat", "282) tiger cat", "283) Persian cat", "284) Siamese cat, Siamese", "285) Egyptian cat", "286) cougar, puma, catamount, mountain lion, painter, panther, Felis concolor", "287) lynx, catamount", "288) leopard, Panthera pardus", "289) snow leopard, ounce, Panthera uncia", "290) jaguar, panther, Panthera onca, Felis onca", "291) lion, king of beasts, Panthera leo", "292) tiger, Panthera tigris", "293) cheetah, chetah, Acinonyx jubatus", "294) brown bear, bruin, Ursus arctos", "295) American black bear, black bear, Ursus americanus, Euarctos americanus", "296) ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus", "297) sloth bear, Melursus ursinus, Ursus ursinus", "298) mongoose", "299) meerkat, mierkat", "300) tiger beetle", "301) ladybug, ladybeetle, lady beetle, ladybird, ladybird beetle", "302) ground beetle, carabid beetle", "303) long-horned beetle, longicorn, longicorn beetle", "304) leaf beetle, chrysomelid", "305) dung beetle", "306) rhinoceros beetle", "307) weevil", "308) fly", "309) bee", "310) ant, emmet, pismire", "311) grasshopper, hopper", "312) cricket", "313) walking stick, walkingstick, stick insect", "314) cockroach, roach", "315) mantis, mantid", "316) cicada, cicala", "317) leafhopper", "318) lacewing, lacewing fly", "319) dragonfly, darning needle, devil's darning needle, sewing needle, snake feeder, snake doctor, mosquito hawk, skeeter hawk", "320) damselfly", "321) admiral", "322) ringlet, ringlet butterfly", "323) monarch, monarch butterfly, milkweed butterfly, Danaus plexippus", "324) cabbage butterfly", "325) sulphur butterfly, sulfur butterfly", "326) lycaenid, lycaenid butterfly", "327) starfish, sea star", "328) sea urchin", "329) sea cucumber, holothurian", "330) wood rabbit, cottontail, cottontail rabbit", "331) hare", "332) Angora, Angora rabbit", "333) hamster", "334) porcupine, hedgehog", "335) fox squirrel, eastern fox squirrel, Sciurus niger", "336) marmot", "337) beaver", "338) guinea pig, Cavia cobaya", "339) sorrel", "340) zebra", "341) hog, pig, grunter, squealer, Sus scrofa", "342) wild boar, boar, Sus scrofa", "343) warthog", "344) hippopotamus, hippo, river horse, Hippopotamus amphibius", "345) ox", "346) water buffalo, water ox, Asiatic buffalo, Bubalus bubalis", "347) bison", "348) ram, tup", "349) bighorn, bighorn sheep, cimarron, Rocky Mountain bighorn, Rocky Mountain sheep, Ovis canadensis", "350) ibex, Capra ibex", "351) hartebeest", "352) impala, Aepyceros melampus", "353) gazelle", "354) Arabian camel, dromedary, Camelus dromedarius", "355) llama", "356) weasel", "357) mink", "358) polecat, fitch, foulmart, foumart, Mustela putorius", "359) black-footed ferret, ferret, Mustela nigripes", "360) otter", "361) skunk, polecat, wood pussy", "362) badger", "363) armadillo", "364) three-toed sloth, ai, Bradypus tridactylus", "365) orangutan, orang, orangutang, Pongo pygmaeus", "366) gorilla, Gorilla gorilla", "367) chimpanzee, chimp, Pan troglodytes", "368) gibbon, Hylobates lar", "369) siamang, Hylobates syndactylus, Symphalangus syndactylus", "370) guenon, guenon monkey", "371) patas, hussar monkey, Erythrocebus patas", "372) baboon", "373) macaque", "374) langur", "375) colobus, colobus monkey", "376) proboscis monkey, Nasalis larvatus", "377) marmoset", "378) capuchin, ringtail, Cebus capucinus", "379) howler monkey, howler", "380) titi, titi monkey", "381) spider monkey, Ateles geoffroyi", "382) squirrel monkey, Saimiri sciureus", "383) Madagascar cat, ring-tailed lemur, Lemur catta", "384) indri, indris, Indri indri, Indri brevicaudatus", "385) Indian elephant, Elephas maximus", "386) African elephant, Loxodonta africana", "387) lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens", "388) giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca", "389) barracouta, snoek", "390) eel", "391) coho, cohoe, coho salmon, blue jack, silver salmon, Oncorhynchus kisutch", "392) rock beauty, Holocanthus tricolor", "393) anemone fish", "394) sturgeon", "395) gar, garfish, garpike, billfish, Lepisosteus osseus", "396) lionfish", "398) abacus", "399) abaya", "400) academic gown, academic robe, judge's robe", "401) accordion, piano accordion, squeeze box", "402) acoustic guitar", "403) aircraft carrier, carrier, flattop, attack aircraft carrier", "404) airliner", "405) airship, dirigible", "406) altar", "407) ambulance", "408) amphibian, amphibious vehicle", "409) analog clock", "410) apiary, bee house", "411) apron", "412) ashcan, trash can, garbage can, wastebin, ash bin, ash-bin, ashbin, dustbin, trash barrel, trash bin", "413) assault rifle, assault gun", "414) backpack, back pack, knapsack, packsack, rucksack, haversack", "415) bakery, bakeshop, bakehouse", "416) balance beam, beam", "417) balloon", "418) ballpoint, ballpoint pen, ballpen, Biro", "419) Band Aid", "420) banjo", "421) bannister, banister, balustrade, balusters, handrail", "422) barbell", "423) barber chair", "424) barbershop", "425) barn", "426) barometer", "427) barrel, cask", "428) barrow, garden cart, lawn cart, wheelbarrow", "429) baseball", "430) basketball", "431) bassinet", "432) bassoon", "433) bathing cap, swimming cap", "434) bath towel", "435) bathtub, bathing tub, bath, tub", "436) beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon", "437) beacon, lighthouse, beacon light, pharos", "438) beaker", "439) bearskin, busby, shako", "440) beer bottle", "441) beer glass", "442) bell cote, bell cot", "443) bib", "444) bicycle-built-for-two, tandem bicycle, tandem", "445) bikini, two-piece", "446) binder, ring-binder", "447) binoculars, field glasses, opera glasses", "448) birdhouse", "449) boathouse", "450) bobsled, bobsleigh, bob", "451) bolo tie, bolo, bola tie, bola", "452) bonnet, poke bonnet", "453) bookcase", "454) bookshop, bookstore, bookstall", "455) bottlecap", "456) bow", "457) bow tie, bow-tie, bowtie", "458) brass, memorial tablet, plaque", "459) brassiere, bra, bandeau", "460) breakwater, groin, groyne, mole, bulwark, seawall, jetty", "461) breastplate, aegis, egis", "462) broom", "463) bucket, pail", "464) buckle", "465) bulletproof vest", "466) bullet train, bullet", "467) butcher shop, meat market", "469) caldron, cauldron", "470) candle, taper, wax light", "471) cannon", "472) canoe", "473) can opener, tin opener", "474) cardigan", "475) car mirror", "476) carousel, carrousel, merry-go-round, roundabout, whirligig", "477) carpenter's kit, tool kit", "478) carton", "479) car wheel", "480) cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM", "481) cassette", "482) cassette player", "483) castle", "484) catamaran", "485) CD player", "486) cello, violoncello", "487) cellular telephone, cellular phone, cellphone, cell, mobile phone", "488) chain", "489) chainlink fence", "490) chain mail, ring mail, mail, chain armor, chain armour, ring armor, ring armour", "491) chain saw, chainsaw", "492) chest", "493) chiffonier, commode", "494) chime, bell, gong", "495) china cabinet, china closet", "496) Christmas stocking", "497) church, church building", "498) cinema, movie theater, movie theatre, movie house, picture palace", "499) cleaver, meat cleaver, chopper", "500) cliff dwelling", "501) cloak", "502) clog, geta, patten, sabot", "503) cocktail shaker", "504) coffee mug", "505) coffeepot", "506) coil, spiral, volute, whorl, helix", "507) combination lock", "508) computer keyboard, keypad", "509) confectionery, confectionary, candy store", "510) container ship, containership, container vessel", "511) convertible", "512) corkscrew, bottle screw", "513) cornet, horn, trumpet, trump", "514) cowboy boot", "515) cowboy hat, ten-gallon hat", "516) cradle", "517) crane", "518) crash helmet", "519) crate", "520) crib, cot", "521) Crock Pot", "522) croquet ball", "523) crutch", "524) cuirass", "525) dam, dike, dyke", "526) desk", "527) desktop computer", "528) dial telephone, dial phone", "529) diaper, nappy, napkin", "530) digital clock", "531) digital watch", "532) dining table, board", "533) dishrag, dishcloth", "534) dishwasher, dish washer, dishwashing machine", "535) disk brake, disc brake", "536) dock, dockage, docking facility", "537) dogsled, dog sled, dog sleigh", "538) dome", "539) doormat, welcome mat", "540) drilling platform, offshore rig", "541) drum, membranophone, tympan", "542) drumstick", "543) dumbbell", "544) Dutch oven", "545) electric fan, blower", "546) electric guitar", "547) electric locomotive", "548) entertainment center", "549) envelope", "550) espresso maker", "551) face powder", "552) feather boa, boa", "553) file, file cabinet, filing cabinet", "554) fireboat", "555) fire engine, fire truck", "556) fire screen, fireguard", "557) flagpole, flagstaff", "558) flute, transverse flute", "559) folding chair", "560) football helmet", "561) forklift", "562) fountain", "563) fountain pen", "564) four-poster", "565) freight car", "566) French horn, horn", "567) frying pan, frypan, skillet", "568) fur coat", "569) garbage truck, dustcart", "570) gasmask, respirator, gas helmet", "571) gas pump, gasoline pump, petrol pump, island dispenser", "572) goblet", "573) go-kart", "574) golf ball", "575) golfcart, golf cart", "576) gondola", "577) gong, tam-tam", "578) gown", "579) grand piano, grand", "580) greenhouse, nursery, glasshouse", "581) grille, radiator grille", "582) grocery store, grocery, food market, market", "583) guillotine", "584) hair slide", "585) hair spray", "586) half track", "587) hammer", "588) hamper", "589) hand blower, blow dryer, blow drier, hair dryer, hair drier", "590) hand-held computer, hand-held microcomputer", "591) handkerchief, hankie, hanky, hankey", "592) hard disc, hard disk, fixed disk", "593) harmonica, mouth organ, harp, mouth harp", "594) harp", "595) harvester, reaper", "596) hatchet", "597) holster", "598) home theater, home theatre", "599) honeycomb", "600) hook, claw", "601) hoopskirt, crinoline", "602) horizontal bar, high bar", "603) horse cart, horse-cart", "604) hourglass", "605) iPod", "606) iron, smoothing iron", "607) jack-o'-lantern", "608) jean, blue jean, denim", "609) jeep, landrover", "610) jersey, T-shirt, tee shirt", "611) jigsaw puzzle", "612) jinrikisha, ricksha, rickshaw", "613) joystick", "614) kimono", "615) knee pad", "616) knot", "617) lab coat, laboratory coat", "618) ladle", "619) lampshade, lamp shade", "620) laptop, laptop computer", "621) lawn mower, mower", "622) lens cap, lens cover", "623) letter opener, paper knife, paperknife", "624) library", "625) lifeboat", "626) lighter, light, igniter, ignitor", "627) limousine, limo", "628) liner, ocean liner", "629) lipstick, lip rouge", "630) Loafer", "631) lotion", "632) loudspeaker, speaker, speaker unit, loudspeaker system, speaker system", "633) loupe, jeweler's loupe", "634) lumbermill, sawmill", "635) magnetic compass", "636) mailbag, postbag", "637) mailbox, letter box", "638) maillot", "639) maillot, tank suit", "640) manhole cover", "641) maraca", "642) marimba, xylophone", "643) mask", "644) matchstick", "645) maypole", "646) maze, labyrinth", "647) measuring cup", "648) medicine chest, medicine cabinet", "649) megalith, megalithic structure", "650) microphone, mike", "651) microwave, microwave oven", "652) military uniform", "653) milk can", "654) minibus", "655) miniskirt, mini", "656) minivan", "657) missile", "658) mitten", "659) mixing bowl", "660) mobile home, manufactured home", "661) Model T", "662) modem", "663) monastery", "664) monitor", "665) moped", "666) mortar", "667) mortarboard", "668) mosque", "669) mosquito net", "670) motor scooter, scooter", "671) mountain bike, all-terrain bike, off-roader", "672) mountain tent", "673) mouse, computer mouse", "674) mousetrap", "675) moving van", "676) muzzle", "677) nail", "678) neck brace", "679) necklace", "680) nipple", "681) notebook, notebook computer", "682) obelisk", "683) oboe, hautboy, hautbois", "684) ocarina, sweet potato", "685) odometer, hodometer, mileometer, milometer", "686) oil filter", "687) organ, pipe organ", "688) oscilloscope, scope, cathode-ray oscilloscope, CRO", "689) overskirt", "690) oxcart", "691) oxygen mask", "692) packet", "693) paddle, boat paddle", "694) paddlewheel, paddle wheel", "695) padlock", "696) paintbrush", "697) pajama, pyjama, pj's, jammies", "698) palace", "699) panpipe, pandean pipe, syrinx", "700) paper towel", "701) parachute, chute", "702) parallel bars, bars", "703) park bench", "704) parking meter", "705) passenger car, coach, carriage", "706) patio, terrace", "707) pay-phone, pay-station", "708) pedestal, plinth, footstall", "709) pencil box, pencil case", "710) pencil sharpener", "711) perfume, essence", "712) Petri dish", "713) photocopier", "714) pick, plectrum, plectron", "715) pickelhaube", "716) picket fence, paling", "717) pickup, pickup truck", "718) pier", "719) piggy bank, penny bank", "720) pill bottle", "721) pillow", "722) ping-pong ball", "723) pinwheel", "724) pirate, pirate ship", "725) pitcher, ewer", "726) plane, carpenter's plane, woodworking plane", "727) planetarium", "728) plastic bag", "729) plate rack", "730) plow, plough", "731) plunger, plumber's helper", "732) Polaroid camera, Polaroid Land camera", "733) pole", "734) police van, police wagon, paddy wagon, patrol wagon, wagon, black Maria", "735) poncho", "736) pool table, billiard table, snooker table", "737) pop bottle, soda bottle", "738) pot, flowerpot", "739) potter's wheel", "740) power drill", "741) prayer rug, prayer mat", "742) printer", "743) prison, prison house", "744) projectile, missile", "745) projector", "746) puck, hockey puck", "747) punching bag, punch bag, punching ball, punchball", "748) purse", "749) quill, quill pen", "750) quilt, comforter, comfort, puff", "751) racer, race car, racing car", "752) racket, racquet", "753) radiator", "754) radio, wireless", "755) radio telescope, radio reflector", "756) rain barrel", "757) recreational vehicle, RV, R.V.", "758) reel", "759) reflex camera", "760) refrigerator, icebox", "761) remote control, remote", "762) restaurant, eating house, eating place, eatery", "763) revolver, six-gun, six-shooter", "764) rifle", "765) rocking chair, rocker", "766) rotisserie", "767) rubber eraser, rubber, pencil eraser", "768) rugby ball", "769) rule, ruler", "770) running shoe", "771) safe", "772) safety pin", "773) saltshaker, salt shaker", "774) sandal", "775) sarong", "776) sax, saxophone", "777) scabbard", "778) scale, weighing machine", "779) school bus", "780) schooner", "781) scoreboard", "782) screen, CRT screen", "783) screw", "784) screwdriver", "785) seat belt, seatbelt", "786) sewing machine", "787) shield, buckler", "788) shoe shop, shoe-shop, shoe store", "789) shoji", "790) shopping basket", "791) shopping cart", "792) shovel", "793) shower cap", "794) shower curtain", "795) ski", "796) ski mask", "797) sleeping bag", "798) slide rule, slipstick", "799) sliding door", "800) slot, one-armed bandit", "801) snorkel", "802) snowmobile", "803) snowplow, snowplough", "804) soap dispenser", "805) soccer ball", "806) sock", "807) solar dish, solar collector, solar furnace", "808) sombrero", "809) soup bowl", "810) space bar", "811) space heater", "812) space shuttle", "813) spatula", "814) speedboat", "815) spider web, spider's web", "816) spindle", "817) sports car, sport car", "818) spotlight, spot", "819) stage", "820) steam locomotive", "821) steel arch bridge", "822) steel drum", "823) stethoscope", "824) stole", "825) stone wall", "826) stopwatch, stop watch", "827) stove", "828) strainer", "829) streetcar, tram, tramcar, trolley, trolley car", "830) stretcher", "831) studio couch, day bed", "832) stupa, tope", "833) submarine, pigboat, sub, U-boat", "834) suit, suit of clothes", "835) sundial", "836) sunglass", "837) sunglasses, dark glasses, shades", "838) sunscreen, sunblock, sun blocker", "839) suspension bridge", "840) swab, swob, mop", "841) sweatshirt", "842) swimming trunks, bathing trunks", "843) swing", "844) switch, electric switch, electrical switch", "845) syringe", "846) table lamp", "847) tank, army tank, armored combat vehicle, armoured combat vehicle", "848) tape player", "849) teapot", "850) teddy, teddy bear", "851) television, television system", "852) tennis ball", "853) thatch, thatched roof", "854) theater curtain, theatre curtain", "855) thimble", "856) thresher, thrasher, threshing machine", "857) throne", "858) tile roof", "859) toaster", "860) tobacco shop, tobacconist shop, tobacconist", "861) toilet seat", "862) torch", "863) totem pole", "864) tow truck, tow car, wrecker", "865) toyshop", "866) tractor", "867) trailer truck, tractor trailer, trucking rig, rig, articulated lorry, semi", "868) tray", "869) trench coat", "870) tricycle, trike, velocipede", "871) trimaran", "872) tripod", "873) triumphal arch", "874) trolleybus, trolley coach, trackless trolley", "875) trombone", "876) tub, vat", "877) turnstile", "878) typewriter keyboard", "879) umbrella", "880) unicycle, monocycle", "881) upright, upright piano", "882) vacuum, vacuum cleaner", "883) vase", "884) vault", "885) velvet", "886) vending machine", "887) vestment", "888) viaduct", "889) violin, fiddle", "890) volleyball", "891) waffle iron", "892) wall clock", "893) wallet, billfold, notecase, pocketbook", "894) wardrobe, closet, press", "895) warplane, military plane", "896) washbasin, handbasin, washbowl, lavabo, wash-hand basin", "897) washer, automatic washer, washing machine", "898) water bottle", "899) water jug", "900) water tower", "901) whiskey jug", "902) whistle", "903) wig", "904) window screen", "905) window shade", "906) Windsor tie", "907) wine bottle", "908) wing", "909) wok", "910) wooden spoon", "911) wool, woolen, woollen", "912) worm fence, snake fence, snake-rail fence, Virginia fence", "913) wreck", "914) yawl", "915) yurt", "916) web site, website, internet site, site", "917) comic book", "918) crossword puzzle, crossword", "919) street sign", "920) traffic light, traffic signal, stoplight", "921) book jacket, dust cover, dust jacket, dust wrapper", "922) menu", "923) plate", "924) guacamole", "925) consomme", "926) hot pot, hotpot", "927) trifle", "928) ice cream, icecream", "929) ice lolly, lolly, lollipop, popsicle", "930) French loaf", "931) bagel, beigel", "932) pretzel", "933) cheeseburger", "934) hotdog, hot dog, red hot", "935) mashed potato", "936) head cabbage", "937) broccoli", "938) cauliflower", "939) zucchini, courgette", "940) spaghetti squash", "941) acorn squash", "942) butternut squash", "943) cucumber, cuke", "944) artichoke, globe artichoke", "945) bell pepper", "946) cardoon", "947) mushroom", "948) Granny Smith", "949) strawberry", "950) orange", "951) lemon", "952) fig", "953) pineapple, ananas", "954) banana", "955) jackfruit, jak, jack", "956) custard apple", "957) pomegranate", "958) hay", "959) carbonara", "960) chocolate sauce, chocolate syrup", "961) dough", "962) meat loaf, meatloaf", "963) pizza, pizza pie", "964) potpie", "965) burrito", "966) red wine", "967) espresso", "968) cup", "969) eggnog", "970) alp", "971) bubble", "972) cliff, drop, drop-off", "973) coral reef", "974) geyser", "975) lakeside, lakeshore", "976) promontory, headland, head, foreland", "977) sandbar, sand bar", "978) seashore, coast, seacoast, sea-coast", "979) valley, vale", "980) volcano", "981) ballplayer, baseball player", "982) groom, bridegroom", "983) scuba diver", "984) rapeseed", "985) daisy", "986) yellow lady's slipper, yellow lady-slipper, Cypripedium calceolus, Cypripedium parviflorum", "987) corn", "988) acorn", "989) hip, rose hip, rosehip", "990) buckeye, horse chestnut, conker", "991) coral fungus", "992) agaric", "993) gyromitra", "994) stinkhorn, carrion fungus", "995) earthstar", "997) bolete", "998) ear, spike, capitulum", "999) toilet tissue, toilet paper, bathroom tissue"]

def interpolate_and_shape(A, B, num_interps):
  """Linearly interpolate between vectors A and B and reshape."""
  interps = interpolate(A, B, num_interps)
  return (interps.transpose(1, 0, *range(2, len(interps.shape)))
                 .reshape(num_samples * num_interps, *interps.shape[2:]))

z_A, z_B = [truncated_z_sample(num_samples, truncation, noise_seed)
            for noise_seed in [noise_seed_A, noise_seed_B]]
y_A, y_B = [one_hot([int(category.split(')')[0])] * num_samples)
            for category in [category_A, category_B]]

z_interp = interpolate_and_shape(z_A, z_B, num_interps)
y_interp = interpolate_and_shape(y_A, y_B, num_interps)

ims = sample(sess, z_interp, y_interp, truncation=truncation)
plt.figure(figsize=(20, 20))
fig=plt.imshow(imgrid(ims, cols=num_interps))
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)

get_name = lambda name: name.split(")")[1].split(",")[0]

cat_a = get_name(category_A)
cat_b = get_name(category_B)
print("The model is merging a {} with a {}.".format(cat_a, cat_b))
print("Look at the sample in the middle. The model is imagining a '{}-{}'".format(cat_a, cat_b))

# Style Transfer.

**Style transfer** is another example of a generative model exhibiting imagination. In style transfer a generative model applies the style present in a **style image** to the content of a **content** image. Let's have a look at an example below.

In [ ]:
# @title Restart the colab kernel

#@markdown Run this cell and wait for the colab kernel to restart before running the next cell.
#@markdown > There will be a green tick in the top right of the screen when it is time to run the next cell.

import os
os._exit(00)

In [ ]:
#@title Set up the style transfer code
#@markdown This will take about two minutes.
# Import files needed for the style transfer.
%%capture
import functools
import os
from google.colab import files
from PIL import Image as PILImage

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
try:
  tf.executing_eagerly()
except:
  "You need to run the cell above and wait for the colab to restart."
  raise

from tensorflow.keras.utils import get_file
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image

from google.colab import widgets
import ipywidgets

# https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_arbitrary_image_stylization.ipynb#scrollTo=lnAv-F3O9fLV

def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), crop_image=True, preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = plt.imread(image_path).astype(np.float32)[np.newaxis, ...]
  if img.max() > 1.0:
    img = img / 255.
  if len(img.shape) == 3:
    img = tf.stack([img, img, img], axis=-1)
  if crop_image:
    img = crop_center(img)
  if image_size is not None:
    img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def plot_images(images, titles=None):
  """Plots two or three images each next to one-another."""
  assert len(images) in [2, 3]
  if titles is None:
    title_sequence = [''] * (len(images) * 2 - 1)
  else:
    title_sequence = [titles[0], '', titles[1]]
  image_sequence = [images[0], image_plus, images[1]]
  if len(images) == 3:
    image_sequence.extend([image_arrow, images[2]])
    title_sequence.extend(['', titles[2]])
  
  fig, ax = plt.subplots(1, len(image_sequence), figsize=(20, 60))
  for i, (img, ttl) in enumerate(zip(image_sequence, title_sequence)):
    ax[i].imshow(img[0])
    ax[i].axis('off')
    ax[i].set_title(ttl)


def plot_input_images(images, titles=None):
  """Plots the the content and style image images."""
  # images is a list [content image, style image]
  # titles is a list [content title, style title]
  if titles is None:
    title_sequence = [''] * (len(images) * 2 - 1)
  else:
    title_sequence = [titles[0], '', titles[1], '']
  image_sequence = [images[0], image_plus, images[1], image_arrow]
  
  fig, ax = plt.subplots(1, len(image_sequence), figsize=(20, 60))
  for i, (img, ttl) in enumerate(zip(image_sequence, title_sequence)):
    ax[i].imshow(img[0])
    ax[i].axis('off')
    ax[i].set_title(ttl)

#@title Preloading images and model
plus_sign_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Plus_font_awesome.svg/200px-Plus_font_awesome.svg.png'
arrow_sign_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/7/71/Arrow_east.svg/200px-Arrow_east.svg.png'
image_plus_temp = load_image(plus_sign_url, None, crop_image=False)
image_arrow_temp = load_image(arrow_sign_url, None, crop_image=False)

# scaling the images down
image_plus = np.zeros([1, 400, 400, 4])
image_plus[:, 100:300, 100:300, :] = image_plus_temp

image_arrow = np.zeros([1, 162, 400, 4])
image_arrow[:, 40:121, 100:300, :] = image_arrow_temp


# load pretrained model
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

def _imshow(im, title=None):
  fig=plt.imshow(im)
  fig.axes.get_xaxis().set_visible(False)
  fig.axes.get_yaxis().set_visible(False)
  if title is not None:
    plt.title(title)
  plt.show()
  return fig

def download_image(inputs=None, filename='stylized_image.jpeg', image_array='None'):
  assert image_array is not None
  img = (255 * image_array.numpy()).astype(np.uint8)
  im = PILImage.fromarray(img)
  im.save(filename)
  files.download(filename) 


In [ ]:
# @title Load your content and style image. { run: "auto" }
#@markdown Select a `content_image` and a `style_image` from the drop down menu, then run the cell below. This will show your `Content Image` and your `Style Image`. An `Apply style transfer` button will appear, click this and see the new `Stylized image` appear!

fixed_choices = {
    "Van Gogh - The Starry Night": "https://www.vangoghgallery.com/img/starry_night_full.jpg",
    "Van Gogh - Self-portrait with grey felt hat": "https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Vincent_van_Gogh_-_Self-portrait_with_grey_felt_hat_-_Google_Art_Project.jpg/988px-Vincent_van_Gogh_-_Self-portrait_with_grey_felt_hat_-_Google_Art_Project.jpg",
    "Frida Kahlo - Viva la Vida, Watermelons": "https://uploads1.wikiart.org/images/magdalena-carmen-frieda-kahlo-y-calder%C3%B3n-de-rivera/viva-la-vida-watermelons.jpg!Large.jpg",
    "Joan Miro - The Smile of the Flamboyant Wings": "https://uploads5.wikiart.org/images/joan-miro/the-smile-of-the-flamboyant-wings.jpg",
    "Jean-Michel Basquiat - Skull": "https://uploads5.wikiart.org/images/jean-michel-basquiat/head.jpg!Large.jpg",
    "Piet Mondrian - Composition No IV": "https://upload.wikimedia.org/wikipedia/commons/thumb/6/66/Composition_No_IV%2C_by_Piet_Mondriaan.jpg/836px-Composition_No_IV%2C_by_Piet_Mondriaan.jpg",
    "Piet Mondrian - Blossoming apple tree": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/03/Blossoming_apple_tree%2C_by_Piet_Mondriaan.jpg/1599px-Blossoming_apple_tree%2C_by_Piet_Mondriaan.jpg",
    "Andy Warhol - Liz Taylor": "https://uploads2.wikiart.org/images/andy-warhol/liz-taylor.jpg!Large.jpg",
    "Georgia O'Keeffe - Blue and Green Music": "https://upload.wikimedia.org/wikipedia/commons/2/26/Blue-green.jpg",
    "Georgia O'Keeffe - Oriental Poppies": "https://upload.wikimedia.org/wikipedia/en/c/ca/Red_Canna_%281924%29_by_Georgia_O%27Keeffe.jpg",
    "Wassily Kandinsky, 1923 - On White II": "https://upload.wikimedia.org/wikipedia/commons/c/c4/Vassily_Kandinsky%2C_1923_-_On_White_II.jpg",
    "Wassily Kandinsky - Several Circles" : "https://i0.wp.com/www.guggenheim.org/wp-content/uploads/1926/01/41.283_ph_web-1.jpg",
    "Patrick Hoesly - Matrix iPhone Background": "https://live.staticflickr.com/5052/5508849065_c312d5cd28_w.jpg",
    }

fixed_content_images = {
    "House": "https://upload.wikimedia.org/wikipedia/commons/thumb/6/67/Strada_Romulus_nr._77_%28B-II-m-B-19616%29.JPG/354px-Strada_Romulus_nr._77_%28B-II-m-B-19616%29.JPG",
    "Corgi Dog": "https://upload.wikimedia.org/wikipedia/commons/1/12/ThreeTimeAKCGoldWinnerPembrookeWelshCorgi.jpg",
    "Cat": "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Felis_catus-cat_on_snow.jpg/220px-Felis_catus-cat_on_snow.jpg",
    "Beach": "https://upload.wikimedia.org/wikipedia/commons/6/62/Makapu%CA%BBu_Beach.JPG",
    "Tree": "https://upload.wikimedia.org/wikipedia/commons/e/eb/Ash_Tree_-_geograph.org.uk_-_590710.jpg"
}

content_image = "Tree" #@param ["House", "Corgi Dog", "Cat", "Beach", "Tree"]

style_image = "Piet Mondrian - Blossoming apple tree" #@param ["Van Gogh - The Starry Night", "Van Gogh - Self-portrait with grey felt hat", "Frida Kahlo - Viva la Vida, Watermelons", "Joan Miro - The Smile of the Flamboyant Wings", "Jean-Michel Basquiat - Skull", "Piet Mondrian - Composition No IV", "Piet Mondrian - Blossoming apple tree", "Andy Warhol - Liz Taylor", "Georgia O'Keeffe - Blue and Green Music", "Georgia O'Keeffe - Oriental Poppies", "Wassily Kandinsky, 1923 - On White II", "Wassily Kandinsky - Several Circles", "Patrick Hoesly - Matrix iPhone Background"] {allow-input: true}

blur_style_image = True

if style_image in fixed_choices:
  style_image = fixed_choices[style_image]

if content_image in fixed_content_images:
  content_image = fixed_content_images[content_image]

content_image = load_image(content_image, None)
style_image = load_image(style_image, (256, 256))
# blurring the style image for a prettier transfer

if blur_style_image:
  style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')

outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
stylized_image = outputs[0]

def show_style_transfer(inputs=None):
  print("Applying style transfer.")
  with grid.output_to(0, 1):
    style_button.disabled = False
    _imshow(stylized_image[0], "Stylized image.")

grid = widgets.Grid(1, 2)
with grid.output_to(0, 0):
  plot_input_images([content_image, style_image], ['Content image', 'Style image'])



style_button = ipywidgets.Button(description="Apply style transfer")
display(style_button)
style_button.on_click(show_style_transfer)

download_stylised_image = functools.partial(download_image, filename='stylized_image.jpeg', image_array=stylized_image[0])
download_button = ipywidgets.Button(description="Download stylised image")
display(download_button)
download_button.on_click(download_stylised_image)


# See how Artist evolves and improves over time.

The process of the Artist and Detective competing against one-another, with the Artist producing more realistic images and the Detective getting better at distinguishing the real images from the fake ones, may be more formally referred to as **training**. Training the Artist to synthesise natural images can take a long time so we have trained one for you.

In this section you will see what kinds of images the Artist learns to generate while it is training and how it improves with more training steps.

In [ ]:
#@title Imports
%%capture
'''
We're reading the pickle files, each pickle file has 18MB and contains:
  dict_keys(['gen_imgs', 'disc_fake', 'IS_mean', 'IS_std', 'FID']) for a given step.
'''
import glob
import dill
import IPython
import requests
from io import BytesIO
import urllib
from matplotlib import pyplot as plt
from google.colab import widgets
from matplotlib import pylab
import numpy as np
from matplotlib import rc
rc('animation', html='jshtml')
import matplotlib.animation as animation

In [ ]:
#@title Read the saved data.
BASE_PATH = 'https://storage.googleapis.com/dm-educational/assets/generative/bigGAN/1'

FOLDERS = ['100040', '100829', '101618', '102407', '103196', '103985', '104774',
 '105563', '107141', '10727', '107929', '108718', '109504', '110293', '111082',
 '111871', '112660', '114238', '115027', '115816', '116604', '117393', '118182',
 '118971', '120548', '121337', '122126', '122915', '12305', '123704', '124493',
 '125282', '126071', '1261', '126860', '128438', '129227', '130016', '130805',
 '13094', '131594', '132383', '133172', '134749', '135538', '136325', '137114',
 '137903', '138691', '13883']

def update_progress_bar(int_percentage):
  IPython.display.clear_output(wait=True)
  bar = '#' * int_percentage + '.' * (100 - int_percentage)
  print('Reading data:', bar, ' %d%%' % int_percentage)

def get_data():
  folders = glob.glob('bigGAN/1/*')
  res = {}
  for i, folder_id in enumerate(FOLDERS):
    try:
      step = int(folder_id)
      path = f'{BASE_PATH}/{folder_id}/gen_ed.pkl'
      print(path)
      res[step] = dill.load(urllib.request.urlopen(path))
    except (ValueError, FileNotFoundError):
      # It could be that some files were not loaded but it shouldn't be a problem.
      pass
    int_percentage = int(i / len(FOLDERS) * 100)
    update_progress_bar(int_percentage)
  return res

res = get_data()
update_progress_bar(100)
sorted_keys = sorted(res.keys())

scores = []
for key in sorted_keys:
  scores.append(res[key]['IS_mean'])

# Check that we at least read some files.
assert len(sorted_keys) >= 10, 'There was some error when loading the requested files.'

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [ ]:
# @title Explore how the Artist evolves over time. { run: "auto" }


#@markdown Slide the "training_percentage" from left to right to see the Artist get better over time.

#@markdown (1) To look at the images generated at the start of training move the slider to the far left. See that the images are not very realistic.

#@markdown (2) To look at the images generated towards the end of training move the slider to the far right. Notice that the images generated are very realistic.

#@markdown With each image we also report the "Detective Prediction". If the detective thinks that the image is real this value will be close to one, while if the detective thinks that the image is fake it will be close to zero.

training_percentage = 2 #@param {type:"slider", min:0, max:100, step:1}
training = min(int(training_percentage / 100. * len(sorted_keys)), len(sorted_keys) - 1)
num_samples = 9

data = []
key = sorted_keys[training]
indices = [(0, 2), (0, 1), (7, 0), (4, 2),
           (2, 2), (2, 6), (3, 7), (4, 3),
           (6, 7), (0, 7), (6, 3), (7, 1),
           (0, 0), (4, 6), (5, 1), (5, 2)]
for n_sample in range(num_samples):
  img = np.clip(res[key]['gen_imgs'], 0, 1)
  i, j = indices[n_sample]
  plt.show()
  data.append(
      {
          'img': np.transpose(img[i][j], (1, 2, 0)),
          'pred': sigmoid(res[key]['disc_fake'][i][j]),
          'is_score': scores[training],
          'step': int(key),
      }
  )


# Plot performance.
plt.figure(figsize=(10, 6))
plt.plot(sorted_keys, scores)
plt.xlabel('Training steps')
plt.ylabel('Performance (Inception Score, higher is better)')
plt.plot([d['step'] for d in data], [d['is_score'] for d in data], 'rx')
plt.show()
print('\n' * 2)


# Plot grid with images.
rows, cols = int(np.sqrt(num_samples)), int(np.sqrt(num_samples))
grid = widgets.Grid(rows, cols)
for i, (row, col) in enumerate(grid):
  pylab.figure(figsize=(4, 4))
  pylab.imshow(data[i]['img'])
  pylab.yticks([])
  pylab.xticks([])
  title = ' ' * 5 + 'Step: %d' % data[i]['step'] + '\n'
  title += 'Detective Prediction: %.2f' % data[i]['pred']
  pylab.title(title)

In [ ]:
# @title Watch how the Artist improves over time.
#@markdown Press play to run the animation. You can run this cell again to see a different image over time.

i, j = np.random.randint(0, 8, size=(2,))

data = []
for index, key in enumerate(sorted_keys):
  if index % (len(sorted_keys) // 10) == 0:
    img = res[key]['gen_imgs']
    plt.show()
    data.append(
        {
            'img': np.transpose(img[i][j], (1, 2, 0)),
            'pred': sigmoid(res[key]['disc_fake'][i][j]),
            'is_score':  scores[index],
            'step': int(key),
        }
    )

fig = plt.figure()
images = []

for i in range(len(data)):
    im = plt.imshow(np.clip(data[i]['img'], 0, 1), animated=True)
    plt.xticks([])
    plt.yticks([])
    images.append([im])

ani = animation.ArtistAnimation(fig, images, interval=1000, blit=True,
                                repeat_delay=1000)
plt.close()
print('Run this cell again to generate new images!')
ani

In the next section, you will have a chance to try training your own Artist and Detective.

# Try it for yourself.

In this section we will show you how to set up the Artist and Detective.

If you were to dig into the code in the cells below you would notice that rather than having an Artist and a Detective we have a **Generator** (aka the Artist) and a **Discriminator** (aka the detective). These are the more "technical" terms for these components.



In [ ]:
# @title Run this cell and wait for the colab to re-start. 
# Since we're using TensorFlow 2 for this part of the colab we need to restart the kernel and install / import dependencies again.
# @markdown You will need to wait for the colab to restart before running the next cell. There will be a green tick in the top right of the screen when it is time to run the next cell.

import os
os._exit(00)

We are going to train a generative model to generate an image similar to one that you draw.

Before we start training the generative model, we first need some data, in this case a single image, to train our generative model on. This image will serve as the **real** example that the:

1.   Generator (Artist) is trying to replicate your artwork.
2.   Discriminator (Detective) is trying to distinguish your drawing from the Generator's output.

Let's capture your drawing for training. Run the cell below and draw something on the canvas that appears? You could draw anything, even a smiley face. 

Once you have drawn the image press the save button.

In [ ]:
#@title Create a canvas to draw an image on.
#@title Run this cell to import things needed to train your own generative model.
import tensorflow as tf
assert tf.executing_eagerly()

from io import BytesIO
from io import StringIO
from PIL import Image
import base64

import IPython
from IPython.display import display
from google.colab import output
import matplotlib.image as mpimg
from matplotlib import pylab
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import PIL
import time
from tensorflow.keras import layers

import uuid
from google.colab import widgets

display(IPython.display.HTML('''
    <div id='maindiv'>
    <canvas id='canvas' width="300" height="300"
    style="border:3px solid #616161;"></canvas>
    </div>
    <button id='clear'>Clear Canvas</button>
    <img id='img'></img>

<script>
TRACK_MOUSE = false

function clearCanvas() {
  var canvas = document.getElementById('canvas');
  var ctxt2d = canvas.getContext("2d");
  ctxt2d.clearRect(0, 0, 300, 300);
}

function mouseDownEvent(mouseEvent) {
  TRACK_MOUSE = true;
}

function mouseUpEvent(mouseEvent) {
  TRACK_MOUSE = false;
}

IMG_DATA = null

function mouseMoveEvent(mouseEvent) {
  if (TRACK_MOUSE) {
    var square = document.createElement("div");
    square.classList.add('square');
    var canvas = document.getElementById('canvas');
    var ctxt2d = canvas.getContext("2d");
    ctxt2d.beginPath();

    var pointerX = mouseEvent.clientX - canvas.offsetLeft;
    var pointerY = mouseEvent.clientY - canvas.offsetTop
    ctxt2d.arc(pointerX, pointerY, 12, 0, 2 * Math.PI);
    ctxt2d.fill();

    console.log(mouseEvent.clientX - canvas.offsetLeft);
    console.log(mouseEvent.clientY - canvas.offsetTop);

  }
}

function saveImage(e) {
  var canvas = document.getElementById('canvas');
  var ctx = canvas.getContext("2d");
  console.log(canvas.offsetLeft);
  var img = ctx.getImageData(0, 0, 300, 300);
  IMG_DATA = canvas.toDataURL();
  return canvas.toDataURL();
}
document.getElementById('clear').addEventListener("click", clearCanvas);

document.getElementById('canvas').addEventListener("mousemove", mouseMoveEvent);
document.getElementById('canvas').addEventListener("mousedown", mouseDownEvent);
document.getElementById('canvas').addEventListener("mouseup", mouseUpEvent);
</script>
'''))

class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

image = None
def do_something():
  global image
  data_url = output.eval_js('saveImage()')
  header, encoded = data_url.split(",", 1)
  data = base64.b64decode(encoded)
  with open("image.png", "wb") as f:
    f.write(data)
    print('Image saved successfully!')

InvokeButton('Save Image', do_something)

In the next cell we are going to look at the image we have captured and resize the image to make it a little bit smaller. This helps the model to learn faster.

In [ ]:
#@title Show the input image and process the image to make it smaller.
def load_image():
  img = mpimg.imread('image.png')
  img = img[:,:,3] # remove unused channels.

  fig, axes = plt.subplots(1, 2)
  axes[0].set_title('Original Image')
  axes[0].imshow(img, cmap='Greys')
  axes[0].get_xaxis().set_visible(False)
  axes[0].axes.get_yaxis().set_visible(False)
  # plt.show()

  input_img = resize(img, (28, 28)) #/ 255.
  axes[1].set_title('Preproccesed image')
  axes[1].imshow(input_img, cmap='Greys')
  axes[1].get_xaxis().set_visible(False)
  axes[1].get_yaxis().set_visible(False)

  return input_img

img = load_image()

Now that we have the data let's create our Artist and Detective.



In [ ]:
#@title Create the Artist and the Detective.
# Based on https://twitter.com/fchollet/status/1250622989541838848

#@title Create the Artist and the Detective.
# Based on https://twitter.com/fchollet/status/1250622989541838848

loss_fn = tf.losses.BinaryCrossentropy(from_logits=True)


class GeneratorModel(tf.keras.Model):
  def __init__(self, dense_units=[32, 128], final_layer=784, latent_dim=100):
    super().__init__()
    self.latent_dim = latent_dim
    self.dense_layers = [tf.keras.layers.Dense(u, activation=tf.nn.relu) for u in dense_units]
    self.final_layer = tf.keras.layers.Dense(final_layer, activation=tf.nn.sigmoid)

  def call(self, x):
    for dense_layer in self.dense_layers:
      x = dense_layer(x)
    return self.final_layer(x)

class DiscriminatorModel(tf.keras.Model):
  def __init__(self, dense_units=[784, 256, 128, 16], final_layer=1):
    super().__init__()
    self.dense_layers = [tf.keras.layers.Dense(u, activation=tf.nn.relu) for u in dense_units]
    self.final_layer = tf.keras.layers.Dense(final_layer, activation=None)

  def call(self, x):
    for dense_layer in self.dense_layers:
      x = dense_layer(x)
    return self.final_layer(x)

class GAN(tf.keras.Model):

  def __init__(self, generator_net, discriminator_net):
    super().__init__()
    self.generator_net = generator_net
    self.discriminator_net = discriminator_net

    self.gen_train_loss = []
    self.disc_train_loss = []

  def compile(self, generator_opt, discriminator_opt, loss_fn):
    super().compile()
    self.generator_opt = generator_opt
    self.discriminator_opt = discriminator_opt
    self.loss_fn = loss_fn
    self.generator_net.compile()
    self.discriminator_net.compile()

  def update_net(self, data, labels, net, optimizer):
    with tf.GradientTape() as tape:
      preds = net(data)
      loss = self.loss_fn(labels, preds)
    grads = tape.gradient(loss, net.trainable_weights)
    optimizer.apply_gradients(zip(grads, net.trainable_weights))
    return loss

  def train_step(self, real_data):
    batch_size = tf.shape(real_data)[0]
    
    # Generate fake data.
    gen_data = self.sample(batch_size=batch_size)

    # Train discriminator.
    data = tf.concat([real_data, gen_data], axis=0)
    labels = tf.concat([tf.ones((batch_size,1)), tf.zeros((batch_size,1))], axis=0)
    discriminator_loss = self.update_net(data, labels,
                                         self.discriminator_net,
                                         self.discriminator_opt)
    self.disc_train_loss.append(discriminator_loss)
    
    # Train generator.
    all_real_labels = tf.ones((batch_size,1))
    with tf.GradientTape() as tape:
      preds = self.discriminator_net(self.sample(batch_size=batch_size))
      generator_loss = self.loss_fn(all_real_labels, preds)
    grads = tape.gradient(generator_loss, self.generator_net.trainable_weights)
    self.generator_opt.apply_gradients(zip(grads, self.generator_net.trainable_weights))
    self.gen_train_loss.append(generator_loss)

    return {'d_loss': discriminator_loss, 'g_loss': generator_loss}

  def sample(self, batch_size=1, z=None):
    if z is None:
      z = tf.random.normal(shape=(batch_size, self.generator_net.latent_dim))
    return self.generator_net(z)

Run the cell below and watch the Artist get better over time. At the start of the game the Artist outputs noise after a few steps the model gets much better and the image resembles the one you drew above.

You can also change the **`number_of_training_steps`** to see how this affects the Artist skills. If the artist trains for very few steps (like 10) you'll see that they are not able to replicate your art, but you should see that when you increase this number it should get better.

In [ ]:
# @title Look at the Artist getting better over time.

number_of_training_steps = 94 #@param {type:"slider", min:10, max:200, step:1}

# Instantiate the artist (generator)
generator_net = GeneratorModel()
# Instantiate the detective (discriminator)
discriminator_net = DiscriminatorModel()

gan_net = GAN(generator_net, discriminator_net)
gan_net.compile(tf.keras.optimizers.Adam(), tf.keras.optimizers.Adam(),
                tf.keras.losses.BinaryCrossentropy(from_logits=True))

# Run the model and show the images.
num_runs = 10
grid = widgets.Grid(2, 5)
for i, (row, col) in enumerate(grid):
  if i > 0:
    gan_net.fit(np.reshape(img, (1, -1)), epochs=number_of_training_steps // num_runs, verbose=False)
  pylab.figure(figsize=(4, 4))
  pylab.imshow(gan_net.sample().numpy().reshape((28, 28)), cmap='Greys')
  pylab.yticks([])
  pylab.xticks([])
  pylab.title('Time: %d' % i)

This generative model that you just trained is only able to generate images that are very similar to your drawing because that's all the data that it has ever seen!

Typically generative models are trained with millions of images and this allows them to exploit patterns in the images to generate many different images, even ones not seen in the training data.